# Setup and Validation

This notebook validates your environment and configuration before running the SLAVV comparison pipeline. It replaces the legacy `validate_setup.py` script.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import subprocess
from pathlib import Path

# Add project root to path
project_root = Path('..').resolve()
sys.path.insert(0, str(project_root))

from source.slavv.dev.validation import Validator

## Configuration

Set your paths below. If you are unsure, the defaults usually work for standard installations.

In [ ]:
# Path to MATLAB executable
# Windows example: "C:\\Program Files\\MATLAB\\R2023a\\bin\\matlab.exe"
matlab_path = "C:\\Program Files\\MATLAB\\R2019a\\bin\\matlab.exe"

# Path to test data volume
test_data = project_root / "data" / "slavv_test_volume.tif"

print(f"Project Root: {project_root}")
print(f"MATLAB Path: {matlab_path}")
print(f"Test Data: {test_data}")

## Run Validation

Execute the validation checks. This looks for:
1. Required Python packages
2. Valid MATLAB installation
3. Necessary external submodules (Vectorization-Public)
4. Test data integrity

In [ ]:
validator = Validator(project_root, matlab_path=matlab_path, test_data_path=str(test_data))

# Run standard checks
validator.check_python_imports()
validator.check_vectorization_public()
validator.check_matlab_executable()
validator.check_test_data()
validator.check_disk_space(project_root / "comparison_output")

# Run specific MATLAB version check (requires launching MATLAB, may take a moment)
print("Checking MATLAB version...")
validator.check_matlab_version()

## MATLAB Functional Test

This runs a small MATLAB script to confirm it can execute code and find the vectorization library.

In [ ]:
def run_matlab_test():
    if not Path(matlab_path).exists():
        print("⚠️ MATLAB executable not found, skipping test.")
        return

    print("Running MATLAB functional test...")
    # MATLAB script content
    matlab_code = """
    fprintf('MATLAB Setup Test\n');
    current_dir = pwd;
    fprintf('Current directory: %s\n', current_dir);
    
    % Check for shared library logic if needed, or just standard path check
    % Here we assume we are running from project root/notebooks, so vectorization
    % code is in ../external/Vectorization-Public
    
    vec_path = fullfile('..', 'external', 'Vectorization-Public', 'vectorize_V200.m');
    if exist(vec_path, 'file') == 2
        fprintf('✓ vectorize_V200.m found at relative path\n');
    elseif exist(fullfile('..', 'legacy', 'Vectorization-Public', 'vectorize_V200.m'), 'file') == 2
        fprintf('✓ vectorize_V200.m found in legacy path\n');
    else
        fprintf('✗ vectorize_V200.m NOT found relative to notebook\n');
    end
    
    exit;
    """
    
    try:
        result = subprocess.run(
            [matlab_path, "-batch", matlab_code],
            capture_output=True, text=True, timeout=60
        )
        print(result.stdout)
        if result.returncode != 0:
            print(f"❌ MATLAB exited with error code {result.returncode}")
            print(result.stderr)
    except Exception as e:
        print(f"❌ Failed to run MATLAB test: {e}")

run_matlab_test()

## Summary Results

In [ ]:
print("Passed Checks:")
for p in validator.passed:
    print(f"✅ {p.message}")

if validator.warnings:
    print("\nWarnings:")
    for w in validator.warnings:
        print(f"⚠️ {w.message}")

if validator.errors:
    print("\nERRORS:")
    for e in validator.errors:
        print(f"❌ {e.message}")
else:
    print("\n🎉 System is ready for comparison runs!")